В колл-центре СЧАСТЬЕ работают операторы, отвечающие на звонки.
К/ц СЧАСТЬЕ передает нам информацию о звонках в следующем виде.
 
В таблицу CALLS к/ц записывает информацию о звонках. Колонки таблицы:
1. date - дата звонка, строка в формате "YYYY-MM-DD"
2. call_id - идентификатор звонка, строка
3. audio_file - имя файла с записью звонка, строка
4. oper_id - идентификатор оператора, ответившего на звонок, строка
 
При этом данные о звонках могут поступать не сразу, а с задержкой в несколько дней,
поэтому ведется таблица DATES, в которую к/ц записывает даты, за которые пришли новые данные о звонках.
В таблице DATES  единственная колонка:
1. date - дата, за которую пришла информация, строка в том же формате "YYYY-MM-DD"
 
Требуется написать код с использованием spark DataFrame API (этот код будут запускать ежедневно),
который дописывает данные в две таблицы:

1. STAT_TOTAL (колонки 1. date, 2. count, 3. stat_date_time)
- количество записей в таблице CALLS за даты из таблицы DATES
- Примечание. stat_date_time - время запуска расчетов, т.е. текущее дата-время
- Примечание. Если за некую дату нет звонков, то НУЖНО создать запись со значением 0 в колонке count

2. STAT_OPER (колонки 1. date, 2. oper_id, 3. count, 4. stat_date_time)
- количество записей в таблице CALLS за даты из таблицы DATES для каждого оператора
- Примечание. Если за некую дату у некого оператора нет звонков, то НЕ НУЖНО создавать запись со значением 0 в колонке count

После добавления данных в описанные выше две таблицы, требуется очистить таблицу DATES
```
pip install pyspark
import pyspark
from pyspark.sql import Row
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from typing import List
from pandas import DataFrame
spark = SparkSession.builder.config("spark.driver.bindAddress", "127.0.0.1").getOrCreate()
dates = spark.createDataFrame([
    Row(date='2022-01-20'),
    Row(date='2022-01-18'),
    Row(date='2022-01-16'),
])
calls = spark.createDataFrame([
    Row(date='2022-01-20',  call_id='ERCERCER', audio_file='rec_12536123.wav',  oper_id=134),
    Row(date='2022-01-20',  call_id='34FEWEC3', audio_file='rec_89372934.wav',  oper_id=134),
    Row(date='2022-01-20',  call_id='ERC34F3E', audio_file='rec_56756775.wav',  oper_id=134),
    Row(date='2022-01-20',  call_id='ERCJNER8', audio_file='rec_32454565.wav',  oper_id=128),
    Row(date='2022-01-20',  call_id='ERLHCRE8', audio_file='rec_34545567.wav',  oper_id=125),
    Row(date='2022-01-20',  call_id='LKECRE9C', audio_file='rec_23434564.wav',  oper_id=125),
    Row(date='2022-01-19',  call_id='LJC8ER24', audio_file='rec_65778978.wav',  oper_id=127),
    Row(date='2022-01-19',  call_id='KJNDFC94', audio_file='rec_34545766.wav',  oper_id=128),
    Row(date='2022-01-19',  call_id='KJDC9833', audio_file='rec_34545656.wav',  oper_id=125),
    Row(date='2022-01-19',  call_id='JHB38743', audio_file='rec_23434545.wav',  oper_id=125),
    Row(date='2022-01-19',  call_id='U7JH76H5', audio_file='rec_56767876.wav',  oper_id=127),
    Row(date='2022-01-18',  call_id='34F345F4', audio_file='rec_56567678.wav',  oper_id=134),
    Row(date='2022-01-18',  call_id='WED34F45', audio_file='rec_34534534.wav',  oper_id=134),
    Row(date='2022-01-18',  call_id='W3D34F56', audio_file='rec_56756767.wav',  oper_id=134),
    Row(date='2022-01-18',  call_id='WF435F55', audio_file='rec_23434534.wav',  oper_id=116),
    Row(date='2022-01-17',  call_id='NKDBUS83', audio_file='rec_13434876.wav',  oper_id=134),
    Row(date='2022-01-17',  call_id='NBE83642', audio_file='rec_13434468.wav',  oper_id=116),
    Row(date='2022-01-17',  call_id='NVID49DF', audio_file='rec_13434111.wav',  oper_id=134),
])
calls.show()
dates.show()
```



In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 23.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=d977166296c8cf35ad60885a09cc015be6a92c084b212072f45a8cb8b62ef6aa
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
import pyspark
from pyspark.sql import Row
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from typing import List
from pandas import DataFrame

spark = SparkSession.builder.config("spark.driver.bindAddress", "127.0.0.1").getOrCreate()
dates = spark.createDataFrame([
    Row(date='2022-01-20'),
    Row(date='2022-01-18'),
    Row(date='2022-01-16'),
])
calls = spark.createDataFrame([
    Row(date='2022-01-20',  call_id='ERCERCER', audio_file='rec_12536123.wav',  oper_id=134),
    Row(date='2022-01-20',  call_id='34FEWEC3', audio_file='rec_89372934.wav',  oper_id=134),
    Row(date='2022-01-20',  call_id='ERC34F3E', audio_file='rec_56756775.wav',  oper_id=134),
    Row(date='2022-01-20',  call_id='ERCJNER8', audio_file='rec_32454565.wav',  oper_id=128),
    Row(date='2022-01-20',  call_id='ERLHCRE8', audio_file='rec_34545567.wav',  oper_id=125),
    Row(date='2022-01-20',  call_id='LKECRE9C', audio_file='rec_23434564.wav',  oper_id=125),
    Row(date='2022-01-19',  call_id='LJC8ER24', audio_file='rec_65778978.wav',  oper_id=127),
    Row(date='2022-01-19',  call_id='KJNDFC94', audio_file='rec_34545766.wav',  oper_id=128),
    Row(date='2022-01-19',  call_id='KJDC9833', audio_file='rec_34545656.wav',  oper_id=125),
    Row(date='2022-01-19',  call_id='JHB38743', audio_file='rec_23434545.wav',  oper_id=125),
    Row(date='2022-01-19',  call_id='U7JH76H5', audio_file='rec_56767876.wav',  oper_id=127),
    Row(date='2022-01-18',  call_id='34F345F4', audio_file='rec_56567678.wav',  oper_id=134),
    Row(date='2022-01-18',  call_id='WED34F45', audio_file='rec_34534534.wav',  oper_id=134),
    Row(date='2022-01-18',  call_id='W3D34F56', audio_file='rec_56756767.wav',  oper_id=134),
    Row(date='2022-01-18',  call_id='WF435F55', audio_file='rec_23434534.wav',  oper_id=116),
    Row(date='2022-01-17',  call_id='NKDBUS83', audio_file='rec_13434876.wav',  oper_id=134),
    Row(date='2022-01-17',  call_id='NBE83642', audio_file='rec_13434468.wav',  oper_id=116),
    Row(date='2022-01-17',  call_id='NVID49DF', audio_file='rec_13434111.wav',  oper_id=134),
])

calls.show()
dates.show()


+----------+--------+----------------+-------+
|      date| call_id|      audio_file|oper_id|
+----------+--------+----------------+-------+
|2022-01-20|ERCERCER|rec_12536123.wav|    134|
|2022-01-20|34FEWEC3|rec_89372934.wav|    134|
|2022-01-20|ERC34F3E|rec_56756775.wav|    134|
|2022-01-20|ERCJNER8|rec_32454565.wav|    128|
|2022-01-20|ERLHCRE8|rec_34545567.wav|    125|
|2022-01-20|LKECRE9C|rec_23434564.wav|    125|
|2022-01-19|LJC8ER24|rec_65778978.wav|    127|
|2022-01-19|KJNDFC94|rec_34545766.wav|    128|
|2022-01-19|KJDC9833|rec_34545656.wav|    125|
|2022-01-19|JHB38743|rec_23434545.wav|    125|
|2022-01-19|U7JH76H5|rec_56767876.wav|    127|
|2022-01-18|34F345F4|rec_56567678.wav|    134|
|2022-01-18|WED34F45|rec_34534534.wav|    134|
|2022-01-18|W3D34F56|rec_56756767.wav|    134|
|2022-01-18|WF435F55|rec_23434534.wav|    116|
|2022-01-17|NKDBUS83|rec_13434876.wav|    134|
|2022-01-17|NBE83642|rec_13434468.wav|    116|
|2022-01-17|NVID49DF|rec_13434111.wav|    134|
+----------+-

In [3]:
# создание таблицы 
schema = StructType([
    StructField("date", StringType(), True),
    StructField("count", IntegerType(), True),
    StructField("stat_date_time", TimestampType(), True)
])
stat_total = spark.createDataFrame([], schema)

# создание таблицы 
schema = StructType([
    StructField("date", StringType(), True),
    StructField("oper_id", StringType(), True),
    StructField("count", IntegerType(), True),
    StructField("stat_date_time", TimestampType(), True)
])
stat_oper = spark.createDataFrame([], schema)
# подсчет количества записей из данных таблиц
calls = calls.withColumn("date", F.to_date("date"))
dates = dates.withColumn("date", F.to_date("date"))

# получение уникальных значений из таблиц
distinct_calls_dates = calls.select("date").distinct()
distinct_dates = dates.select("date").distinct()

count_by_date = calls.join(dates, "date") \
    .groupBy("date") \
    .agg(F.count("*").alias("count")) \
    .withColumn("stat_date_time", F.current_timestamp())

# получение даты, которые есть в таблице dates, но которых нет в таблице calls
missing_dates = distinct_dates.subtract(distinct_calls_dates)

# создание записей со значением 0 в колонке count для отсутствующих дат
missing_count_by_date = missing_dates \
    .withColumn("count", F.lit(0)) \
    .withColumn("stat_date_time", F.current_timestamp())
    
# добавление их в таблицу stat_total
stat_total = count_by_date.union(missing_count_by_date)

# подсчет количества записей в таблице calls за даты из таблицы DATES для каждого оператора
count_by_oper_date = calls.join(dates, "date") \
    .groupBy("date", "oper_id") \
    .agg(F.count("*").alias("count")) \
    .withColumn("stat_date_time", F.current_timestamp())

# добавление данных в таблицу stat_oper 
stat_oper = stat_oper.union(count_by_oper_date)
stat_total.show()
stat_oper.show()

+----------+-----+--------------------+
|      date|count|      stat_date_time|
+----------+-----+--------------------+
|2022-01-20|    6|2023-04-12 04:42:...|
|2022-01-18|    4|2023-04-12 04:42:...|
|2022-01-16|    0|2023-04-12 04:42:...|
+----------+-----+--------------------+

+----------+-------+-----+--------------------+
|      date|oper_id|count|      stat_date_time|
+----------+-------+-----+--------------------+
|2022-01-20|    134|    3|2023-04-12 04:42:...|
|2022-01-20|    128|    1|2023-04-12 04:42:...|
|2022-01-20|    125|    2|2023-04-12 04:42:...|
|2022-01-18|    134|    3|2023-04-12 04:42:...|
|2022-01-18|    116|    1|2023-04-12 04:42:...|
+----------+-------+-----+--------------------+



In [4]:
# очистка таблицы dates 
dates = spark.createDataFrame([], dates.schema)
dates.show()

+----+
|date|
+----+
+----+

